# Implementing Linear Support Vector Classifier from Scratch


Import Section 


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from  sklearn.model_selection import train_test_split

Lets work on the cancer dataset

In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!mkdir cancerdataset
!kaggle datasets download -d amandam1/breastcancerdataset
!unzip breastcancerdataset.zip -d cancerdataset

Saving kaggle.json to kaggle.json
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 9.80MB/s]
Archive:  breastcancerdataset.zip
  inflating: cancerdataset/BRCA.csv  


Lets Load the dataset

In [3]:
cancerdata = pd.read_csv('/content/cancerdataset/BRCA.csv')
cancerdata.head()

,Patient_ID,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
1,TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
2,TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive
3,TCGA-D8-A1XR,56.0,FEMALE,0.345090,-0.21147,-0.19304,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,25-Jan-17,12-Jul-17,Alive
4,TCGA-BH-A0BF,56.0,FEMALE,0.221550,1.90680,0.52045,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,06-May-17,27-Jun-19,Dead


Lets do some preprocessing

In [4]:
# drop duplicates
print("data before duplicate removal : ",cancerdata.shape[0])
cancerdata = cancerdata.drop_duplicates()
print("data after duplicate remomval : ",cancerdata.shape[0])


data before duplicate removal :  341
data after duplicate remomval :  335


In [5]:
# lets only take few features which are important
cancerdata = cancerdata.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,15]]
cancerdata.head()


,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Patient_Status
0,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,Alive
1,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,Dead
2,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Alive
3,56.0,FEMALE,0.345090,-0.21147,-0.19304,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,Alive
4,56.0,FEMALE,0.221550,1.90680,0.52045,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Dead


In [6]:
# null value analysis
print(cancerdata.isnull().sum())
print("data before null remomval : ",cancerdata.shape[0])
cancerdata = cancerdata.dropna()
print("data after null remomval : ",cancerdata.shape[0])
    

Age                1
Gender             1
Protein1           1
Protein2           1
Protein3           1
Protein4           1
Tumour_Stage       1
Histology          1
ER status          1
PR status          1
HER2 status        1
Surgery_type       1
Patient_Status    14
dtype: int64
data before null remomval :  335
data after null remomval :  321


In [7]:
# lets encode the categorical features
X = cancerdata.iloc[:,:-1]
y = cancerdata.iloc[:,-1]


In [ ]:
y.shape

(321,)

In [8]:
features_with_noencoding = X.iloc[:,[0,2,3,4,5]]
feature_with_ordinal_encoding = X.iloc[:,[6]]
feature_with_one_hot_encoding = X.iloc[:,[1,7,8,9,10,11]]
feature_with_ordinal_encoding.Tumour_Stage = feature_with_ordinal_encoding.Tumour_Stage.map({'I':1,"II":2,"III":3})

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
feature_with_one_hot_encoding = enc.fit_transform(feature_with_one_hot_encoding).toarray()
feature_with_one_hot_encoding = pd.DataFrame(feature_with_one_hot_encoding,columns = enc.get_feature_names_out())
feature_with_one_hot_encoding


,Gender_FEMALE,Gender_MALE,Histology_Infiltrating Ductal Carcinoma,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,ER status_Positive,PR status_Positive,HER2 status_Negative,HER2 status_Positive,Surgery_type_Lumpectomy,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
317,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
318,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
319,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [10]:
feature_with_one_hot_encoding.shape

(321, 13)

In [11]:
X = pd.concat([features_with_noencoding.reset_index(), feature_with_ordinal_encoding.reset_index(),feature_with_one_hot_encoding.reset_index()], axis=1)
X.shape
y = y.map({"Alive":0,"Dead":1})

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(224, 22)
(97, 22)
(224,)
(97,)


In [16]:
class SVM:

	def __init__(self, kernel = "linear", learning_rate = 1e-4, regularization_strength = 1.0, max_iter = 2000):
 
		self.num_feats = int
		self.train_size = int
		self.weights = np.array 
		self.y_train = np.array 
		self.input_matrix = np.array

		self.kernel = kernel
		self.kernel_matrix = np.array
		self.support_vectors = np.array
		self.learning_rate = learning_rate 	#Learning rate for gradient descent
		self.regularization_strength = regularization_strength 	#Regularization parameter, to control bias-variance tradeoff
		self.max_iter = max_iter	#Maximum Number of iterations to run gradient descent
		self.cost_threshold = 0.1 * learning_rate  #stopping criterion for gradient descent

	def fit(self, X, y):

		""" Adjust weights to training data """

		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]
		self.input_matrix = np.append(X, np.ones(self.train_size).reshape(-1, 1), axis = 1)   #Add Column with Ones for intercept term 
		self.y_train = np.where(y == 0, -1, 1)
		self.weights = np.zeros(self.num_feats + 1) #Extra +1 for the intercept


		#optimize weights
		prev_cost = float("inf")
		for i in range(self.max_iter):
			cost = self._update_weights()
			
			if i%100 ==0 or i == self.max_iter:
				print("Cost after {} iterations is: {}".format(i, cost))
			if abs(prev_cost -cost) < self.cost_threshold*prev_cost:
				print("Cost after {} iterations is: {}".format(i, cost))
				break
			prev_cost = cost

	def _update_weights(self):

		"""
			Cost Function:
				l(w) = sum(max(0, 1 - y(wX + b))) + (lambda/2)(||w||)^2
				First Term is Hinge Loss, Second is Regularization term
			Gradient:
			    delta_w = dl/dw = (1/n) * ( if y(wX+b) < 1: -yX + lambda*w else: lambda*w )
			Gradient Descent
				w = w - (learning_rate * delta_w)
		"""
		y_pred = (self.weights * self.input_matrix).sum(axis = 1) # y_pred = wX+b

		dist = (1 - (self.y_train * y_pred))
		dist[dist < 0] = 0
		hinge_loss = np.sum(dist)/self.train_size
		regularization_term = (1/2) * self.regularization_strength * (np.dot(self.weights, self.weights))
		cost = hinge_loss  + regularization_term

		delta_w =  self.regularization_strength * (self.weights)

		for y, yhat, X in zip(self.y_train, y_pred, self.input_matrix):

			if y*yhat < 1:
				delta_w -= y*X

		delta_w /= self.train_size
		self.weights = self.weights - (self.learning_rate * delta_w)

		return cost

	def predict(self, X):

		""" Make predictions on given X using trained model """

		size = X.shape[0]
		X = np.append(X, np.ones(size).reshape(-1, 1), axis = 1)

		y_pred = np.sign((self.weights * X).sum(axis = 1))

		y_pred[np.where(y_pred == -1)] = 0.0

		return y_pred 

In [24]:
svm_clf = SVM(learning_rate = 1e-5, regularization_strength = 2,max_iter = 3000)

svm_clf.fit(X_train, y_train)

print('*'*30)
print("Coefficients: {}".format(svm_clf.weights[:-1]))
print("Intercept: {}".format(svm_clf.weights[-1]))
print('*'*30)
print("Train Accuracy: {}".format(accuracy_score(y_train, svm_clf.predict(X_train))))
print("Test Accuracy: {}".format(accuracy_score(y_test, svm_clf.predict(X_test))))
print('*'*30)

Cost after 0 iterations is: 1.0
Cost after 100 iterations is: 0.5105810407662766
Cost after 200 iterations is: 0.48750164311171434
Cost after 300 iterations is: 0.4848089775170802
Cost after 380 iterations is: 0.4844580014054351
******************************
Coefficients: [-2.12208998e-04 -1.59275468e-02  4.52292965e-05 -2.14007522e-04
  2.65344366e-04  1.37275509e-04 -2.12208998e-04 -8.63331015e-04
 -6.90817283e-04 -4.69990498e-04  1.69193573e-05 -2.62896291e-04
 -1.74327021e-04 -1.58478292e-05 -4.53071141e-04 -4.53071141e-04
 -3.27358937e-04 -1.25712203e-04 -1.50711341e-04 -1.19952961e-04
 -1.14729600e-04 -6.76772386e-05]
Intercept: -0.0004530711405398863
******************************
Train Accuracy: 0.7901785714285714
Test Accuracy: 0.8041237113402062
******************************
